In [1]:
import cv2
thres = 0.45 # Threshold to detect object

import pyttsx3
import numpy as np
import requests

# ESP32 URL
URL = "http://192.168.137.86"
AWB = True

# Face recognition and opencv setup
cap = cv2.VideoCapture(URL + ":81/stream")


def set_resolution(url: str, index: int=1, verbose: bool=False):
    try:
        if verbose:
            resolutions = "10: UXGA(1600x1200)\n9: SXGA(1280x1024)\n8: XGA(1024x768)\n7: SVGA(800x600)\n6: VGA(640x480)\n5: CIF(400x296)\n4: QVGA(320x240)\n3: HQVGA(240x176)\n0: QQVGA(160x120)"
            print("available resolutions\n{}".format(resolutions))

        if index in [10, 9, 8, 7, 6, 5, 4, 3, 0]:
            requests.get(url + "/control?var=framesize&val={}".format(index))
        else:
            print("Wrong index")
    except:
        print("SET_RESOLUTION: something went wrong")

def set_quality(url: str, value: int=1, verbose: bool=False):
    try:
        if value >= 10 and value <=63:
            requests.get(url + "/control?var=quality&val={}".format(value))
    except:
        print("SET_QUALITY: something went wrong")

def set_awb(url: str, awb: int=1):
    try:
        awb = not awb
        requests.get(url + "/control?var=awb&val={}".format(1 if awb else 0))
    except:
        print("SET_QUALITY: something went wrong")
    return awb

if __name__ == '__main__':
         set_resolution(URL, index=8)
# cap = cv2.VideoCapture(0)
# cap.set(3,640)
# cap.set(4,480)
# cap.set(10,70)

classNames= [ ]
classFile = "coco.names"
with open(classFile,"rt") as f:
    classNames = f.read().rstrip("\n").split("\n")

configPath = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
weightsPath = "frozen_inference_graph.pb"

net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

engine = pyttsx3.init()

while True:
    if cap.isOpened():
            ret, img = cap.read()

    if ret:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
    if ((cv2.waitKey(0)) == ord('q')):
        cv2.destroyAllWindows()
        break

    else:
        
        ret,img = cap.read()
        classIds, confs, bbox = net.detect (img, confThreshold=thres)
        print(classIds,bbox)
        # key = cv2. waitKey(1)
        if len (classIds) != 0 :
            for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
                cv2.rectangle(img,box,color=(0,255,0),thickness=2)
                cv2.putText(img,classNames[classId-1].upper(),(box[0]+10,box[1]+30),
                cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                engine.say("a"+str(classNames[classId - 1]) + "is in front of you")
                engine.runAndWait()
                cv2.putText(img,str(round(confidence*100,2)),(box[0]+200,box[1]+30),
                cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                break

    cv2.imshow('A-EYE  Vision', img)
    cv2.waitKey(1)

[10  1] [[149  95 167 145]
 [149  95 167 145]]
[ 1 41  1] [[130  91 188 149]
 [191 117 129 122]
 [135   0 184 239]]
[1] [[269 161 369 318]]
[1 1] [[274 165 364 310]
 [358 200 282 210]]
[ 1 33] [[278 119 360 354]
 [ 49 108 149 125]]
[ 1  1 33 77 33] [[293  88 345 350]
 [289 165 343 202]
 [ 52  99 152 123]
 [392 336 164 103]
 [403 340 164 100]]
